In [1]:
pip install opencv-python opencv-python-headless cvzone


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install mediapipe


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install numpy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import cv2
import cvzone
from cvzone.PoseModule import PoseDetector
import os

# Initialize video capture with higher resolution
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Set width
cap.set(4, 720)   # Set height

detector = PoseDetector()

# Fixed ratio for resizing the shirt image
fixedRatio = 262 / 190
shirtRatioHeightWidth = 581 / 440  # Height-to-width ratio of the shirt image

# Define absolute path to resources
resource_path = r"C:\Users\shaik\Downloads\Myntra-HackerRamp-WeForShe2024-Team-CODE4CAUSE-main\Myntra-HackerRamp-WeForShe2024-Team-CODE4CAUSE-main"
shirtFolderPath = os.path.join(resource_path, "Resources", "Shirts")  # Folder containing shirts

# List available shirts (sorted for order consistency)
listShirts = sorted(os.listdir(shirtFolderPath))
imageNumber = 0  # Current shirt index

# Hand movement tracking
prevRightX = 0
prevLeftX = 0

# Store previous shoulder width to detect movement (for auto-fit)
prevShoulderWidth = None

# Create OpenCV window (Resizable)
cv2.namedWindow("Virtual Try-On", cv2.WINDOW_NORMAL)  # Allows manual resizing

while True:
    success, img = cap.read()
    if not success:
        break

    img = detector.findPose(img)
    lmList, bboxInfo = detector.findPosition(img, bboxWithHands=True, draw=False)  # Enable hand detection

    if lmList:
        lm11 = lmList[11][0:2]  # Left Shoulder
        lm12 = lmList[12][0:2]  # Right Shoulder
        rightHand = lmList[16][0]  # Right hand X position
        leftHand = lmList[15][0]  # Left hand X position

        # Calculate shoulder width to determine distance
        shoulderWidth = abs(lm11[0] - lm12[0])

        # Adjust shirt size dynamically based on distance
        if prevShoulderWidth is not None:
            scaleFactor = shoulderWidth / prevShoulderWidth
        else:
            scaleFactor = 1  # Default scale factor

        prevShoulderWidth = shoulderWidth  # Update previous width

        # Calculate the width of the shirt dynamically
        widthOfShirt = int(shoulderWidth * fixedRatio * scaleFactor)

        if widthOfShirt > 0:
            shirtPath = os.path.join(shirtFolderPath, listShirts[imageNumber])
            imgShirt = cv2.imread(shirtPath, cv2.IMREAD_UNCHANGED)

            if imgShirt is not None:
                imgShirt = cv2.resize(imgShirt, (widthOfShirt, int(widthOfShirt * shirtRatioHeightWidth)))
                currentScale = (lm11[0] - lm12[0]) / 190
                offset = int(44 * currentScale), int(48 * currentScale)

                overlay_x = lm12[0] - offset[0]
                overlay_y = max(0, lm12[1] - offset[1])  # Prevent negative values

                img = cvzone.overlayPNG(img, imgShirt, (overlay_x, overlay_y))

        # Detect swipe gestures for changing shirts
        if rightHand - prevRightX > 50:  # Swipe right → Next shirt
            if imageNumber < len(listShirts) - 1:
                imageNumber += 1
                print("Next shirt")
        elif leftHand - prevLeftX < -50:  # Swipe left → Previous shirt
            if imageNumber > 0:
                imageNumber -= 1
                print("Previous shirt")

        # Update previous hand positions
        prevRightX = rightHand
        prevLeftX = leftHand

    # Show the video output (Resizable window)
    cv2.imshow("Virtual Try-On", img)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Next shirt
Previous shirt
Next shirt
Next shirt
Next shirt
Next shirt
Next shirt
